In [2]:
from openimages.download import download_dataset
import ssl
import glob
import os
import shutil
import xml.etree.ElementTree as ET

In [3]:
#pip install --upgrade certifi
download_dataset(".", ["Human head",], annotation_format="pascal", limit=5000)

2021-01-10  10:47:44 INFO Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-01-10  10:47:44 INFO NumExpr defaulting to 8 threads.
2021-01-10  10:47:48 INFO Downloading 5000 train images for class 'human head'
100%|██████████| 5000/5000 [05:08<00:00, 16.21it/s]
2021-01-10  10:52:56 INFO Creating 5000 train annotations (pascal) for class 'human head'
100%|██████████| 5000/5000 [00:10<00:00, 484.38it/s]


{'human head': {'images_dir': '.\\human head\\images',
  'annotations_dir': '.\\human head\\pascal'}}

In [4]:
def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    box_list = []

    for boxes in root.iter('object'):

        filename = root.find('filename').text

        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)

        box = [xmin, ymin, xmax, ymax]
        box_list.append(box)

    return filename, box_list

In [5]:
xml_files = glob.glob("./human head/pascal/*.xml")
trainfile = open("./Source_Images/Training_Images/data_train.txt","w")
testfile = open("./Source_Images/Test_Images/data_test.txt","w")
trainsize = 4500

cnt = 0;
for filename in sorted(xml_files):
    #print (filename)
    if cnt < trainsize:
            image_name, box_list = read_content(filename)
            trainfile.write(os.getcwd() + "/Source_Images/Training_Images/" + image_name)
            shutil.copy("./human head/images/"+ image_name, "./Source_Images/Training_Images/"+ image_name)
    else:
        image_name, box_list = read_content(filename)
        testfile.write(os.getcwd()+"/Source_Images/Test_Images/"+image_name)
        shutil.copy("./human head/images/"+ image_name, "./Source_Images/Test_Images/"+ image_name)
        
    for box in box_list:
        if cnt < trainsize:
            trainfile.write( " " + str(box[0]) + "," + str(box[1]) + "," + str(box[2]) + "," + str(box[3]) + ",0")
        else:
            testfile.write( " " + str(box[0]) + "," + str(box[1]) + "," + str(box[2]) + "," + str(box[3]) + ",0")
            
    if cnt < trainsize:
            trainfile.write( "\n")
    else:
        testfile.write( "\n")
        
    cnt = cnt + 1
trainfile.close()
trainfile.close()